#### We seperate our coding into 3 files :
#### 1_DataFrame_merging_final
#### 2_first_model_final
#### 3_second_model_final
in the first one we make a DataFrame, in second one there is our coding for first model and second model is in third file 

# 1- Data Wrangling
In this part we are focusing on reformatting our data to be able merge our files in one DataFram, we will do it step by step.

#### The first step is to check our data in both files, 'price_demand_data.csv' and 'weather_data.csv'

In [1]:
import pandas as pd

In [2]:
# upload the file
price = pd.read_csv('price_demand_data.csv')
price.head()

,REGION,SETTLEMENTDATE,TOTALDEMAND,PRICECATEGORY
0,VIC1,1/01/2021 0:30,4179.21,LOW
1,VIC1,1/01/2021 1:00,4047.76,LOW
2,VIC1,1/01/2021 1:30,3934.70,LOW
3,VIC1,1/01/2021 2:00,3766.45,LOW
4,VIC1,1/01/2021 2:30,3590.37,LOW


##### We need to summarize data as we don’t need to know the demands for different hours in a day. So, we remove all the hours from SETTLEMENT column. We just need daily bases data.

In [3]:
# Because there is white space between date and time, we first strip all hours then strip space. you will see why later!
price['SETTLEMENTDATE'] = price['SETTLEMENTDATE'].str.rstrip(':0123456789') 
price['SETTLEMENTDATE'] = price['SETTLEMENTDATE'].str.rstrip(' ')

In [4]:
price.head()

,REGION,SETTLEMENTDATE,TOTALDEMAND,PRICECATEGORY
0,VIC1,1/01/2021,4179.21,LOW
1,VIC1,1/01/2021,4047.76,LOW
2,VIC1,1/01/2021,3934.70,LOW
3,VIC1,1/01/2021,3766.45,LOW
4,VIC1,1/01/2021,3590.37,LOW


##### Let's check what we have in REGION column? 

In [5]:
price['REGION'].describe()

count     11664
unique        1
top        VIC1
freq      11664
Name: REGION, dtype: object

##### All the data is for VIC1 so we drop REGION column. 

##### If we put Date as index in both DataFrame we can merge them together later, before doing that we rename SETTLEMENTDATE to Date. 

In [6]:
price = price.drop(['REGION'] , axis = 1) # We don't need 'REGION' column as all of them are VIC1.
price = price.rename(columns = {'SETTLEMENTDATE': 'Date'})
price = price.set_index('Date') # set index so we can merge two dataframe.
price.head()

,TOTALDEMAND,PRICECATEGORY
Date,,
1/01/2021,4179.21,LOW
1/01/2021,4047.76,LOW
1/01/2021,3934.70,LOW
1/01/2021,3766.45,LOW
1/01/2021,3590.37,LOW


##### To find maximum daily price category, we replace 'LOW' to 1, 'MEDIUM' to 2, 'HIGH' to 3 and 'EXTREME' to 4 to find maximum price category, then we will replace to original

In [7]:
# Changing categorical data to numeric so we can find maximum price category per day, we will change numeric to categories later
price['PRICECATEGORY'] = price['PRICECATEGORY'].replace(to_replace = "LOW",value = 1)
price['PRICECATEGORY'] = price['PRICECATEGORY'].replace(to_replace = "MEDIUM",value = 2)
price['PRICECATEGORY'] = price['PRICECATEGORY'].replace(to_replace = "HIGH",value = 3)
price['PRICECATEGORY'] = price['PRICECATEGORY'].replace(to_replace = "EXTREME",value = 4)

In [8]:
price.head()

,TOTALDEMAND,PRICECATEGORY
Date,,
1/01/2021,4179.21,1
1/01/2021,4047.76,1
1/01/2021,3934.70,1
1/01/2021,3766.45,1
1/01/2021,3590.37,1


### Now we groupby by date and aggregate as 'TOTALDEMAND': 'max’, 'PRICECATEGORY': 'max', we think that for total demand we should consider maximum demand not adding all the demand during every day. We change name of TOTALDEMAND to Max_Demand

In [9]:
price1 = price.groupby('Date').agg({'TOTALDEMAND': 'max' , 'PRICECATEGORY': 'max'})
price1 = price1.rename(columns = {'TOTALDEMAND': 'Max_Demand', 'PRICECATEGORY': 'Max_Price_Category'})

price1

,Max_Demand,Max_Price_Category
Date,,
1/01/2021,5019.64,1
1/02/2021,5228.29,2
1/03/2021,5225.37,1
1/04/2021,5807.02,2
1/05/2021,5261.09,2
...,...,...
9/04/2021,5688.63,2
9/05/2021,5222.89,2
9/06/2021,7224.91,2


##### Now we change the values of 'PRICECATEGORY' to original ones:

In [10]:
price1 = price1.replace(to_replace = 1,value = "LOW")
price1 = price1.replace(to_replace = 2,value = "MEDIUM")
price1 = price1.replace(to_replace = 3,value = "HIGH")
price1 = price1.replace(to_replace = 4,value = "EXTREME")
price1.head()

,Max_Demand,Max_Price_Category
Date,,
1/01/2021,5019.64,LOW
1/02/2021,5228.29,MEDIUM
1/03/2021,5225.37,LOW
1/04/2021,5807.02,MEDIUM
1/05/2021,5261.09,MEDIUM


# Important note: 
## We changed:
### TOTALDEMAND to Max_Demand 
### PRICECATEGORY to Max_Price_Category and
### SETTLEMENTDATE to Date.
### Our data now is daily with maximum demand and maximum price category per day. In next file(2_first_model_final) we need to model max demand based on weather data and in another file (3-second_model_final) we need to model maximum price category per day based on weather information.



##### Let's have a look at second DataFrame about weather:

In [11]:
weather = pd.read_csv('weather_data.csv')
weather.head()

,Date,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,9am Temperature (°C),...,9am cloud amount (oktas),9am wind direction,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa)
0,1/01/2021,15.6,29.9,0.0,2.8,9.3,NNE,31.0,13:14,19.2,...,6,N,2,1018.8,28.1,43,5.0,E,13,1015.3
1,2/01/2021,18.4,29.0,0.0,9.4,1.3,NNW,30.0,8:22,23.3,...,7,NNW,17,1013.3,28.7,38,7.0,SW,4,1008.5
2,3/01/2021,17.0,26.2,12.6,4.8,7.1,WSW,33.0,17:55,18.3,...,8,WSW,4,1007.7,23.5,59,4.0,SSW,2,1005.2
3,4/01/2021,16.0,18.6,2.6,3.8,0.0,SSE,41.0,16:03,16.2,...,8,SSE,11,1010.0,18.2,82,8.0,SSW,17,1011.0
4,5/01/2021,15.9,19.1,11.2,1.0,0.0,SSE,35.0,11:02,17.2,...,8,SSE,13,1012.5,18.2,82,8.0,SSE,19,1013.3


##### We calculate Average temperature and Range per day and add to DataFrame, might be help us in our model:

In [12]:
weather['Average temperature (°C)'] = (weather['Minimum temperature (°C)'] + weather['Maximum temperature (°C)']) / 2
weather['Temperature range (°C)'] = weather['Maximum temperature (°C)'] - weather['Minimum temperature (°C)']

##### We put Date as index in both DataFrame so we can merge them together later. 

In [13]:
weather = weather.set_index('Date')
weather.head()

,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,9am Temperature (°C),9am relative humidity (%),...,9am wind speed (km/h),9am MSL pressure (hPa),3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa),Average temperature (°C),Temperature range (°C)
Date,,,,,,,,,,,,,,,,,,,,,
1/01/2021,15.6,29.9,0.0,2.8,9.3,NNE,31.0,13:14,19.2,77.0,...,2,1018.8,28.1,43,5.0,E,13,1015.3,22.75,14.3
2/01/2021,18.4,29.0,0.0,9.4,1.3,NNW,30.0,8:22,23.3,52.0,...,17,1013.3,28.7,38,7.0,SW,4,1008.5,23.70,10.6
3/01/2021,17.0,26.2,12.6,4.8,7.1,WSW,33.0,17:55,18.3,100.0,...,4,1007.7,23.5,59,4.0,SSW,2,1005.2,21.60,9.2
4/01/2021,16.0,18.6,2.6,3.8,0.0,SSE,41.0,16:03,16.2,98.0,...,11,1010.0,18.2,82,8.0,SSW,17,1011.0,17.30,2.6
5/01/2021,15.9,19.1,11.2,1.0,0.0,SSE,35.0,11:02,17.2,96.0,...,13,1012.5,18.2,82,8.0,SSE,19,1013.3,17.50,3.2


# Now we merge both DataFrame 

In [14]:
result = pd.merge(weather,price1, left_index = True,right_index = True, how = 'inner')
result

,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,9am Temperature (°C),9am relative humidity (%),...,3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa),Average temperature (°C),Temperature range (°C),Max_Demand,Max_Price_Category
Date,,,,,,,,,,,,,,,,,,,,,
1/01/2021,15.6,29.9,0.0,2.8,9.3,NNE,31.0,13:14,19.2,77.0,...,28.1,43,5.0,E,13,1015.3,22.75,14.3,5019.64,LOW
2/01/2021,18.4,29.0,0.0,9.4,1.3,NNW,30.0,8:22,23.3,52.0,...,28.7,38,7.0,SW,4,1008.5,23.70,10.6,4964.35,LOW
3/01/2021,17.0,26.2,12.6,4.8,7.1,WSW,33.0,17:55,18.3,100.0,...,23.5,59,4.0,SSW,2,1005.2,21.60,9.2,4503.31,LOW
4/01/2021,16.0,18.6,2.6,3.8,0.0,SSE,41.0,16:03,16.2,98.0,...,18.2,82,8.0,SSW,17,1011.0,17.30,2.6,4764.18,LOW
5/01/2021,15.9,19.1,11.2,1.0,0.0,SSE,35.0,11:02,17.2,96.0,...,18.2,82,8.0,SSE,19,1013.3,17.50,3.2,4800.64,LOW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27/08/2021,4.6,13.6,0.0,1.2,3.8,SSW,15.0,12:18,7.7,88.0,...,12.8,65,7.0,SSE,7,1017.4,9.10,9.0,6769.89,EXTREME
28/08/2021,5.3,17.8,0.0,1.6,9.6,N,39.0,13:14,9.1,80.0,...,17.4,31,3.0,NNW,24,1013.5,11.55,12.5,5716.32,MEDIUM
29/08/2021,9.1,16.2,0.6,6.4,4.3,NNE,33.0,1:50,10.6,79.0,...,12.8,84,7.0,S,6,1010.4,12.65,7.1,6227.89,EXTREME


#### Price1 has 244 rows and weather has 243 when we merge inner, we delete one row from price which is data for 1/9/20

# 2- Data Cleaning
In this part we try to find errors like NaN in our DataFram.

##### Let's check how many NaNs we have in our DataFrame: 

In [15]:
result.isna().sum()

Minimum temperature (°C)             1
Maximum temperature (°C)             1
Rainfall (mm)                        2
Evaporation (mm)                     0
Sunshine (hours)                     0
Direction of maximum wind gust       3
Speed of maximum wind gust (km/h)    3
Time of maximum wind gust            3
9am Temperature (°C)                 1
9am relative humidity (%)            1
9am cloud amount (oktas)             0
9am wind direction                   1
9am wind speed (km/h)                1
9am MSL pressure (hPa)               2
3pm Temperature (°C)                 0
3pm relative humidity (%)            0
3pm cloud amount (oktas)             1
3pm wind direction                   0
3pm wind speed (km/h)                0
3pm MSL pressure (hPa)               1
Average temperature (°C)             1
Temperature range (°C)               1
Max_Demand                           0
Max_Price_Category                   0
dtype: int64

In [16]:
result.loc[:].isna().sum().sum()

23

##### So we have 23 NaNs in our DataFrame, Lets find these 23 locate in howmany rows: 

In [17]:
len(result) - len(result.dropna())

6

##### 6 rows has NaN we could replace NaNs with mean or find similar rows and find the best match but 6 0f 243 looks acceptable to discard:

In [18]:
result = result.dropna(axis = 0)
result

,Minimum temperature (°C),Maximum temperature (°C),Rainfall (mm),Evaporation (mm),Sunshine (hours),Direction of maximum wind gust,Speed of maximum wind gust (km/h),Time of maximum wind gust,9am Temperature (°C),9am relative humidity (%),...,3pm Temperature (°C),3pm relative humidity (%),3pm cloud amount (oktas),3pm wind direction,3pm wind speed (km/h),3pm MSL pressure (hPa),Average temperature (°C),Temperature range (°C),Max_Demand,Max_Price_Category
Date,,,,,,,,,,,,,,,,,,,,,
1/01/2021,15.6,29.9,0.0,2.8,9.3,NNE,31.0,13:14,19.2,77.0,...,28.1,43,5.0,E,13,1015.3,22.75,14.3,5019.64,LOW
2/01/2021,18.4,29.0,0.0,9.4,1.3,NNW,30.0,8:22,23.3,52.0,...,28.7,38,7.0,SW,4,1008.5,23.70,10.6,4964.35,LOW
3/01/2021,17.0,26.2,12.6,4.8,7.1,WSW,33.0,17:55,18.3,100.0,...,23.5,59,4.0,SSW,2,1005.2,21.60,9.2,4503.31,LOW
4/01/2021,16.0,18.6,2.6,3.8,0.0,SSE,41.0,16:03,16.2,98.0,...,18.2,82,8.0,SSW,17,1011.0,17.30,2.6,4764.18,LOW
5/01/2021,15.9,19.1,11.2,1.0,0.0,SSE,35.0,11:02,17.2,96.0,...,18.2,82,8.0,SSE,19,1013.3,17.50,3.2,4800.64,LOW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26/08/2021,5.5,12.9,1.2,2.4,1.0,SE,31.0,12:54,9.2,82.0,...,12.2,55,7.0,SSE,13,1020.3,9.20,7.4,7126.76,EXTREME
27/08/2021,4.6,13.6,0.0,1.2,3.8,SSW,15.0,12:18,7.7,88.0,...,12.8,65,7.0,SSE,7,1017.4,9.10,9.0,6769.89,EXTREME
28/08/2021,5.3,17.8,0.0,1.6,9.6,N,39.0,13:14,9.1,80.0,...,17.4,31,3.0,NNW,24,1013.5,11.55,12.5,5716.32,MEDIUM


##### Let's check data type in each column

In [19]:
print(result.dtypes)

Minimum temperature (°C)             float64
Maximum temperature (°C)             float64
Rainfall (mm)                        float64
Evaporation (mm)                     float64
Sunshine (hours)                     float64
Direction of maximum wind gust        object
Speed of maximum wind gust (km/h)    float64
Time of maximum wind gust             object
9am Temperature (°C)                 float64
9am relative humidity (%)            float64
9am cloud amount (oktas)               int64
9am wind direction                    object
9am wind speed (km/h)                 object
9am MSL pressure (hPa)               float64
3pm Temperature (°C)                 float64
3pm relative humidity (%)              int64
3pm cloud amount (oktas)             float64
3pm wind direction                    object
3pm wind speed (km/h)                 object
3pm MSL pressure (hPa)               float64
Average temperature (°C)             float64
Temperature range (°C)               float64
Max_Demand

##### 9am wind speed (km/h) and 3pm wind speed (km/h) are object but they should be numerical.

In [20]:
result['9am wind speed (km/h)'] 

Date
1/01/2021        2
2/01/2021       17
3/01/2021        4
4/01/2021       11
5/01/2021       13
              ... 
26/08/2021    Calm
27/08/2021    Calm
28/08/2021       7
29/08/2021      13
30/08/2021      15
Name: 9am wind speed (km/h), Length: 237, dtype: object

##### Clam! I see in first row the wind speed is 2(km/h), we can replace Calm with 0 or 1, if there is any 1 in our DataFrame we change Calm to 0 but the minimum is 2 so we change it to 1.

In [21]:
result=result.replace(to_replace="Calm",value='1') # as other data is string we change Calm to string of 1

In [22]:
result['9am wind speed (km/h)'] = result['9am wind speed (km/h)'].astype(float)
result['3pm wind speed (km/h)'] = result['3pm wind speed (km/h)'].astype(float) 

In [23]:
print(result.dtypes)

Minimum temperature (°C)             float64
Maximum temperature (°C)             float64
Rainfall (mm)                        float64
Evaporation (mm)                     float64
Sunshine (hours)                     float64
Direction of maximum wind gust        object
Speed of maximum wind gust (km/h)    float64
Time of maximum wind gust             object
9am Temperature (°C)                 float64
9am relative humidity (%)            float64
9am cloud amount (oktas)               int64
9am wind direction                    object
9am wind speed (km/h)                float64
9am MSL pressure (hPa)               float64
3pm Temperature (°C)                 float64
3pm relative humidity (%)              int64
3pm cloud amount (oktas)             float64
3pm wind direction                    object
3pm wind speed (km/h)                float64
3pm MSL pressure (hPa)               float64
Average temperature (°C)             float64
Temperature range (°C)               float64
Max_Demand

#### Let's save our phenomena! as 'result.csv' file.

In [24]:
result.to_csv('result.csv')

# Now we clean and wrankling our Data and make our DataFrame, We have Max_Demand and Max_Price_Category columns for each days
# In next file (2_first_model_final) we will define our First Model.